In [1]:
from typing import List, Dict, Tuple, Optional, Union, Any

import pandas as pd
import asyncio
from prompt_template_collection import PromptTemplate
from doraemon import Doraemon
from relaxed_fda import RelaxedFDA
from metrics_collection import MetricsHelper


task_name = "HotpotQA"
hp_m = None # -> SoTs 4,6,8,10,12
hp_k = 4 # -> cluster 1,3,4,5,7
case_study = False
conc = True
logger = Doraemon.get_logger(name=__name__, logfile="relaxed_FDA_on_hotpot_qa.log")
df=pd.read_pickle('/kaggle/input/building-hotpotqa-sots-dataset/sots_df.pkl')
df.rename(columns={'question':'query', 'reason':'r_s', 'ground_truth':'g_t','temperature':'t_p'}, inplace=True)
if hp_m:
    df = df.groupby('query').head(hp_m).reset_index(drop=True)
df.to_pickle('sots_df.pkl')

grouped_data = RelaxedFDA.prepare_dataset(file_path='/kaggle/working/sots_df.pkl', logger=logger, context=True)
D: List[Dict] = PromptTemplate.sot_construct_inter_hotpotqa(number=5)
op_system_prompt = RelaxedFDA.get_optimize_system_prompt(task_name)
encoder=RelaxedFDA.get_encoder()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 5.1 MB/s eta 0:00:00


2025-07-28 12:41:34.931493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753706495.185306      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753706495.259950      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-28 12:41:53,347 INFO Loaded dataset from /kaggle/working/sots_df.pkl with shape (11298, 8)
2025-07-28 12:41:53,348 INFO In r_s mode.
2025-07-28 12:41:53,429 INFO Dataset preparation completed with 1295 entries.


In [2]:
if case_study:
    grouped_data = grouped_data.iloc[[16]]

tasks=grouped_data.to_dict(orient='records')

In [3]:
if conc:
    tasks=grouped_data.to_dict(orient='records')

    batch_size = 20
    
    async def process_batches(tasks, *args, **kwargs):
        results = []
        for i in range(0, len(tasks), batch_size):
            batch = tasks[i:i + batch_size]
            result_pd = await RelaxedFDA.async_evaluate(
                batch,
                *args,
                **kwargs
            )
            results.append(result_pd)
            await asyncio.sleep(5)
        # Concatenate all DataFrames into one
        return pd.concat(results, ignore_index=True)
    

    result_pd: pd.DataFrame = await process_batches(
        tasks,
        logger,
        op_system_prompt,
        encoder,
        D,
        enable_logger_rs=False,
        ablation='all',
        K=hp_k,
        T=3,
        max_tokens=300,
        max_concurrent=2
    )

else:    
    result_pd: pd.DataFrame = RelaxedFDA.evaluate(
        grouped_data,
        logger,
        op_system_prompt,
        encoder,
        D,
        enable_logger_rs=False,
        ablation='wo_wt',
        K=hp_k,
        T=2,
        max_tokens=300,
    )

Evaluating rows:   0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
2025-07-28 12:42:06,742 INFO Initialized client for provider gpt3
/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_kmeans.py:1033: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return self.fit(X, sample_weight=sample_weight).labels_
2025-07-28 12:42:09,895 INFO Cluster 2: No items found
2025-07-28 12:42:10,293 INFO Cluster 3: No items foun

In [4]:
MetricsHelper.evaluate(result_pd, logger)

2025-07-28 15:05:59,956 INFO Exact Match: 61.31%
2025-07-28 15:05:59,957 INFO Average F1 Score: 75.97%
2025-07-28 15:05:59,958 INFO Accuracy: 61.31%
